In [ ]:
import sys

IN_COLAB = "google.colab" in sys.modules
if IN_COLAB:
    !pip install tianshou gym wandb

In [3]:
import tianshou as ts
import gym
from tianshou.utils.net.common import Net
import tianshou.utils.net.discrete as discrete
import torch
from torch.utils.tensorboard import SummaryWriter
from tianshou.utils import WandbLogger

In [15]:
logger = WandbLogger()
writer = SummaryWriter("logs/lunar-lander")
logger.load(writer)

D:\mambaforge\envs\rlenv\lib\site-packages\notebook\utils.py:280: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  return LooseVersion(v) >= LooseVersion(check)
wandb: Currently logged in as: darthreca. Use `wandb login --relogin` to force relogin
D:\mambaforge\envs\rlenv\lib\site-packages\wandb\sdk\lib\ipython.py:58: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display


D:\mambaforge\envs\rlenv\lib\site-packages\wandb\sdk\lib\import_hooks.py:246: DeprecationWarning: Deprecated since Python 3.4. Use importlib.util.find_spec() instead.
  loader = importlib.find_loader(fullname, path)


In [4]:
env = gym.make("LunarLander-v2")
train_env = ts.env.DummyVectorEnv([lambda: gym.make("LunarLander-v2") for _ in range(10)])
test_env = ts.env.DummyVectorEnv([lambda: gym.make("LunarLander-v2") for _ in range(2)])

In [25]:
state_shape = env.observation_space.shape or env.observation_space.n
actions_n = env.action_space.shape or env.action_space.n

actor = discrete.Actor(Net(state_shape, hidden_sizes=[32, 32]), action_shape=actions_n)
actor_optim = torch.optim.Adam(actor.parameters(), lr=3e-4)

critic_1 = discrete.Critic(Net(state_shape, actions_n, hidden_sizes=[32, 32]), last_size=actions_n)
critic_1_optim = torch.optim.Adam(critic_1.parameters(), lr=3e-4)

critic_2 = discrete.Critic(Net(state_shape, actions_n, hidden_sizes=[32, 32]), last_size=actions_n)
critic_2_optim = torch.optim.Adam(critic_2.parameters(), lr=3e-4)

In [27]:
critic_1

Critic(
  (preprocess): Net(
    (model): MLP(
      (model): Sequential(
        (0): Linear(in_features=8, out_features=32, bias=True)
        (1): ReLU()
        (2): Linear(in_features=32, out_features=32, bias=True)
        (3): ReLU()
        (4): Linear(in_features=32, out_features=4, bias=True)
      )
    )
  )
  (last): MLP(
    (model): Sequential(
      (0): Linear(in_features=4, out_features=1, bias=True)
    )
  )
)

In [28]:
dqn = ts.policy.DiscreteSACPolicy(
    actor, actor_optim,
    critic_1, critic_1_optim,
    critic_2, critic_2_optim,
    tau=0.005, gamma=0.9,
)

In [29]:
train_collector = ts.data.Collector(dqn, train_env, ts.data.VectorReplayBuffer(20000, 10), exploration_noise=True)
test_collector = ts.data.Collector(dqn, test_env, exploration_noise=True)

In [17]:
def train_fn(epoch, step):
    pass

In [31]:
ts.trainer.offpolicy_trainer(
    dqn, train_collector, test_collector, max_epoch=10, batch_size=128,
    step_per_epoch=10000, step_per_collect=10,
    train_fn=train_fn,
    episode_per_test=10, stop_fn=lambda rew: rew >= env.spec.reward_threshold,
    logger=logger
)

Epoch #1:   0%|          | 10/10000 [00:00<00:16, 593.97it/s]


RuntimeError: index 1 is out of bounds for dimension 1 with size 1

In [32]:
train_env.close()
test_env.close()